In [2]:
import time
import requests
import re
import urllib3
# from selenium.webdriver import Chrome
# from selenium.webdriver.chrome.options import Options
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
from urllib.request import urlretrieve



key_dict ={#'自然': {
#             '1': 'X516aa109bb0e995fb9183da331f6dc5d',
#             '2': 'X62a42f940e0b5bde76b52d568ab6223e',
#             '3': 'X7320baef39aaee92e5e318bd3c776820',
#             '4': 'X14515c7fb2acddc0ceff4b71a723e981',
#             '5': 'Xe03f656f8639ecabd3a388bcef0d7939',
#             },
            '美女':{
            '1': 'X5af501ad81a0e430a6ff7493a63138ac',
            '2': 'Xe0c006e4f64bd8b1bff27ce8a65ab266',
            '3': 'X92f47019ac8a6cf10cb1ad1a38891319',
            '4': 'Xb98b3d2fd78e179d3b443f0783ebdf31',
            '5': 'X52e56263d4d29f9f23d1717835370ed5',
            },
            '女神':{
            '1': 'X698d34b89d5b5de7d2835aee69226aca',
            '2': 'Xc1ba67ba1b4d6682192e754ac9a44cf8',
            '3': 'X06b6d6243a2e64529321a27084a006db',
            '4': 'X78cab11ee5abc3a9b80a7edb88235349',
            '5': 'X933ca9ef652977e47c0150d4830e5b9a',
            },
            '佳丽':{
            '1': 'X0cae1e72816209fe589095ae79b8a204',
            '2': 'X6b0c6bb9d778ec82f3704c9a0cb5a600',
            '3': 'X149556b23d31d73f59556eb23550f65d',
            '4': 'X2266a435aee75b4bf77ddbf691be602f',
            '5': 'X5b159571a8e38238fef9cfbb8be70e9b',
            },
            '美人':{
            '1': 'Xddb1ab07da0e44f7c2a947994dac51d9',
            '2': 'X3551e3240967b57830719ea20991521d',
            '3': 'X9e665956f91e029d34cc6a2371d4ad1e',
            '4': 'X688b11f53a12b6e4628a5c9c97205a35',
            '5': 'X6aea4098ea0ef55e6d9d31256f3f8a43',
            },
          }
folder = f"C:\\Users\\liuqi\\Desktop\\xiaohongshu\\app_image"

def get_picture(url):
    jpgs = []
    driver.get(url)
    time.sleep(0)
    img_links = driver.find_elements_by_xpath("//span[@class='inner']")
    video_links = driver.find_elements_by_xpath("//video[@class='videocontent']")
    for link in img_links:
        link = link.get_attribute('style')
        if re.match('.*/1080/.*',link):
            jpg = link.replace('background-image: url(','https:').replace(');','').replace('\"','')
            print(jpg)
            jpgs.append(jpg)
    
    for link in video_links:
        link = link.get_attribute('poster')
        jpg = link
        print(jpg)
        jpgs.append(jpg)
    return(jpgs)

path = r'C:\Users\liuqi\Desktop\xiaohongshu\app_title\detail_list.xlsx'
data = pd.read_excel(path)
driver = webdriver.Chrome()
start = time.time()
for key, page_dict in key_dict.items():
#     print(old_list)
    for page, value in page_dict.items():
        with open('小红书url.txt', 'r', encoding='utf-8')as f:
            r = f.read().replace('\ufeff', '')
            old_list = r.split('\n')
        url = 'https://www.xiaohongshu.com/fe_api/burdock/weixin/v2/search/notes?'
        headers = {
            'Host': 'www.xiaohongshu.com',
            'Connection': 'keep-alive',
            'Authorization': 'wxmp.11eaca89-052c-4f2c-b06f-6b0ad2dcb919',
            'Device-Fingerprint': 'WHJMrwNw1k/Ex1XMOL3qsQGpSqDxzKY1XYrZUBzScgQX11Jsw9sHBnECb61vhUQ+sRV2uSJGCx8EeHTToqXEPh0dXyEbgJ1dqdCW1tldyDzmauSxIJm5Txg==1487582755342',
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36 MicroMessenger/7.0.9.501 NetType/WIFI MiniProgramEnv/Windows WindowsWechat',
            'X-Sign': value,
            'content-type': 'application/json',
            'Referer': 'https://servicewechat.com/wxffc08ac7df482a27/383/page-frame.html',
            'Accept-Encoding': 'gzip, deflate, br',
        }
        params = {
            'keyword': key,
            'sortBy': 'general',
            'page': page,
            'pageSize': '20',
            'needGifCover': 'true',
        }
        res = requests.get(url, headers=headers, params=params, verify=False).text
        print(res)
        res_dict = json.loads(res)
        notes = res_dict['data']['notes']
        for note in notes:
            id = note['id']
            title = note['title']
            likes = note['likes']
            public_time = note['time']
            user_id = note['user']['id']
            user_name = note['user']['nickname']
            print(id)
            detail_url = 'https://www.xiaohongshu.com/discovery/item/' + id
            print(detail_url)
            if detail_url in old_list:
                print('链接已存在。')
                continue
            else:
                with open('小红书url.txt', 'a', encoding='utf-8')as w:
                    w.write('\n')
                    w.write(detail_url)
                image_urls = get_picture(detail_url)
                new = pd.DataFrame({'page_id':id,
                                    'page_url':detail_url,
                                    'title':title,
                                    'likes':likes,
                                    'public_time':public_time,
                                    'user_id':user_id,
                                    'user_name':user_name,
                                    'get_time':time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
                                   },index = [1])
                data = data.append(new,ignore_index = True)
                for i,jpg in zip(range(len(image_urls)),image_urls):
                    urlretrieve(jpg, f'{folder}\\{id}_{i}.jpg')
                continue
data.to_excel(path, index = False)
cost_time = time.time()-start
print(cost_time)

C:\Users\liuqi\Anaconda3\lib\site-packages\urllib3\connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.xiaohongshu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


{"code":0,"success":true,"data":{"notes":[{"id":"5f757476000000000101f88c","title":"性感美女湿发浴巾诱惑居家写真","type":"normal","likes":325,"isLiked":false,"cover":{"url":"http://ci.xiaohongshu.com/1c54e961-6ade-3e96-a2c1-2ae9ed249c5d?imageView2/2/w/540/format/jpg","gifUrl":"","width":1024,"height":1365,"fileId":"1c54e961-6ade-3e96-a2c1-2ae9ed249c5d"},"time":"2020-10-01 14:17","user":{"id":"5dae54620000000001001ff9","image":"https://img.xiaohongshu.com/avatar/5f37878f621c920001f4950c.jpg@80w_80h_90q_1e_1c_1x.jpg","nickname":"美女之家","redOfficialVerifyType":0,"redOfficialVerifyShowIcon":false,"officialVerified":false}},{"id":"5ed2ee850000000001003088","title":"我爱美女","type":"normal","likes":1579,"isLiked":false,"cover":{"url":"http://ci.xiaohongshu.com/14ee6733-b27b-3be0-b53a-0805587dbd51?imageView2/2/w/540/format/jpg","gifUrl":"","width":1280,"height":1705,"fileId":"14ee6733-b27b-3be0-b53a-0805587dbd51"},"time":"2020-05-31 07:38","user":{"id":"5bb178e5b71c9800015cdc41","image":"https://img.xiaohongsh

C:\Users\liuqi\Anaconda3\lib\site-packages\urllib3\connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.xiaohongshu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


{"code":0,"success":true,"data":{"notes":[{"id":"5de646f6000000000100176f","title":"以前的美女有模有样，现在的美女一模一样！","type":"video","likes":1632,"isLiked":false,"cover":{"url":"http://ci.xiaohongshu.com/abe1763f-f830-3ec5-b880-62ec97bcedde?imageView2/2/w/540/format/jpg","gifUrl":"http://sg.xiaohongshu.com/d38eb64f-89c3-3a36-89da-3b6847ec09fa_gif_w320","width":576,"height":1024,"fileId":"abe1763f-f830-3ec5-b880-62ec97bcedde"},"time":"2019-12-03 19:28","user":{"id":"5d95c5ab000000000100b559","image":"https://img.xiaohongshu.com/avatar/5da1e8455b45330001aa121d.jpg@80w_80h_90q_1e_1c_1x.jpg","nickname":"妞妞","redOfficialVerifyType":0,"redOfficialVerifyShowIcon":false,"officialVerified":false}},{"id":"5a6064567ee0a922fcc30965","title":"网红大美女也爱zara","type":"normal","likes":755,"isLiked":false,"cover":{"url":"http://ci.xiaohongshu.com/8d719602-a444-4e34-9372-50cccda66106?imageView2/2/w/540/format/jpg","gifUrl":"","width":962,"height":1280,"fileId":"8d719602-a444-4e34-9372-50cccda66106"},"time":"2018-01-18

https://ci.xiaohongshu.com/51600ee1-c99f-56f4-9263-c75b3dfe3ce6?imageView2/2/w/1080/format/jpg
5a6064567ee0a922fcc30965
https://www.xiaohongshu.com/discovery/item/5a6064567ee0a922fcc30965
链接已存在。
5ee7677f0000000001001bf1
https://www.xiaohongshu.com/discovery/item/5ee7677f0000000001001bf1
链接已存在。
5f0e8c0e000000000100356b
https://www.xiaohongshu.com/discovery/item/5f0e8c0e000000000100356b
链接已存在。
5ea26268000000000100134e
https://www.xiaohongshu.com/discovery/item/5ea26268000000000100134e
链接已存在。
5ee2e9d7000000000100514c
https://www.xiaohongshu.com/discovery/item/5ee2e9d7000000000100514c
链接已存在。
5f5da377000000000100bd92
https://www.xiaohongshu.com/discovery/item/5f5da377000000000100bd92
链接已存在。
5f118835000000000101df36
https://www.xiaohongshu.com/discovery/item/5f118835000000000101df36
链接已存在。
5e80a43d0000000001006305
https://www.xiaohongshu.com/discovery/item/5e80a43d0000000001006305
链接已存在。
5f0d9bd6000000000100606e
https://www.xiaohongshu.com/discovery/item/5f0d9bd6000000000100606e
链接已存在。
5f19a

...

{"code":0,"success":true,"data":{"notes":[{"id":"5f26715e0000000001004aef","title":"当小猫变成美人","type":"video","likes":5119,"isLiked":false,"cover":{"url":"http://ci.xiaohongshu.com/0e619633f17b23ce733b5ed3ed512ce37662c645?imageView2/2/w/540/format/jpg","gifUrl":"http://sg.xiaohongshu.com/895727104f09d9445683c639007a65e623dc9579_gif_w320","width":576,"height":1024,"fileId":"0e619633f17b23ce733b5ed3ed512ce37662c645"},"time":"2020-08-02 15:55","user":{"id":"5eda12210000000001000abf","image":"https://img.xiaohongshu.com/avatar/5eda12900796a200016c33c8.jpg@80w_80h_90q_1e_1c_1x.jpg","nickname":"你的小橘猫","redOfficialVerifyType":0,"redOfficialVerifyShowIcon":false,"officialVerified":false}},{"id":"5f61b6620000000001001fb6","title":"优雅｜冻龄美人-李英爱","type":"normal","likes":725,"isLiked":false,"cover":{"url":"http://ci.xiaohongshu.com/6b41a955-b759-394e-8e19-4466bb509c8b?imageView2/2/w/540/format/jpg","gifUrl":"","width":900,"height":1200,"fileId":"6b41a955-b759-394e-8e19-4466bb509c8b"},"time":"2020-09-

https://ci.xiaohongshu.com/00a8a6de-c3f1-546c-ab71-5fcc7e5feae1?imageView2/2/w/1080/format/jpg
https://ci.xiaohongshu.com/8b8098c1-c219-51dc-af40-1738dee12df0?imageView2/2/w/1080/format/jpg
https://ci.xiaohongshu.com/7915d085-d0ae-52da-b7ad-906b7a5433eb?imageView2/2/w/1080/format/jpg
https://ci.xiaohongshu.com/9627573d-6d0b-5a03-9fc3-3f1a82ea5a50?imageView2/2/w/1080/format/jpg
https://ci.xiaohongshu.com/61df777d-9dcc-5757-99a7-ff667ca9a539?imageView2/2/w/1080/format/jpg
https://ci.xiaohongshu.com/4d8d1bf7-b59f-5599-936a-69bcb4bfa1aa?imageView2/2/w/1080/format/jpg
https://ci.xiaohongshu.com/5e50645a-223e-53da-b3ac-4ebfa72f956c?imageView2/2/w/1080/format/jpg
https://ci.xiaohongshu.com/3fe1bb6b-231b-5873-a65b-0b099e540366?imageView2/2/w/1080/format/jpg
https://ci.xiaohongshu.com/b37583b6-005a-5beb-ba83-716c3576a7ef?imageView2/2/w/1080/format/jpg
5f5e1e400000000001006e03
https://www.xiaohongshu.com/discovery/item/5f5e1e400000000001006e03
链接已存在。
5f75b9640000000001000f52
https://www.xiaohong

C:\Users\liuqi\Anaconda3\lib\site-packages\urllib3\connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.xiaohongshu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


{"code":0,"success":true,"data":{"notes":[{"id":"5f2f8f4b0000000001005f4a","title":"100位时尚博主安利 迪士尼在逃公主anastasi","type":"normal","likes":6848,"isLiked":false,"cover":{"url":"http://ci.xiaohongshu.com/5b4fc087-50a3-34f9-8d14-8e562cf0e429?imageView2/2/w/540/format/jpg","gifUrl":"","width":1279,"height":1705,"fileId":"5b4fc087-50a3-34f9-8d14-8e562cf0e429"},"time":"2020-08-09 13:53","user":{"id":"5b4da7454eacab18218dc5e7","image":"https://img.xiaohongshu.com/avatar/5f1aecdb399d40000153a0c2.jpg@80w_80h_90q_1e_1c_1x.jpg","nickname":"熬夜少女","redOfficialVerifyType":0,"redOfficialVerifyShowIcon":false,"officialVerified":false}},{"id":"5f72cff80000000001003785","title":"为什么现在没有大美人了？？","type":"normal","likes":5332,"isLiked":false,"cover":{"url":"http://ci.xiaohongshu.com/315c3785-05f0-38e7-813d-0c95fe22875e?imageView2/2/w/540/format/jpg","gifUrl":"","width":1080,"height":1047,"fileId":"315c3785-05f0-38e7-813d-0c95fe22875e"},"time":"2020-09-29 14:11","user":{"id":"5c6984a50000000010039ea8","image":"

https://ci.xiaohongshu.com/3611e419-b829-4477-71b3-5bfe97f27816?imageView2/2/w/1080/format/jpg
5f395c3500000000010038d9
https://www.xiaohongshu.com/discovery/item/5f395c3500000000010038d9
链接已存在。
5f2d1219000000000101e985
https://www.xiaohongshu.com/discovery/item/5f2d1219000000000101e985
链接已存在。
5f3cdf49000000000100271f
https://www.xiaohongshu.com/discovery/item/5f3cdf49000000000100271f
链接已存在。
5f112a54000000000101d973
https://www.xiaohongshu.com/discovery/item/5f112a54000000000101d973
链接已存在。
5f31498e000000000101fa37
https://www.xiaohongshu.com/discovery/item/5f31498e000000000101fa37
链接已存在。
5f7c68e2000000000101d131
https://www.xiaohongshu.com/discovery/item/5f7c68e2000000000101d131
链接已存在。
5f3a5d03000000000101f2ee
https://www.xiaohongshu.com/discovery/item/5f3a5d03000000000101f2ee
链接已存在。
5f6b333c0000000001004f33
https://www.xiaohongshu.com/discovery/item/5f6b333c0000000001004f33
链接已存在。
5f841f01000000000100bcc5
https://www.xiaohongshu.com/discovery/item/5f841f01000000000100bcc5
链接已存在。
5f157